In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import joblib

In [3]:
from sklearn.metrics import root_mean_squared_log_error


In [4]:
import pandas as pd
from utils.preprocess import treat_dataset_pandas_init, build_preprocessing_pipeline,INIT_NUMERICAL_COLS, numerical_features, PredictionTransformer

from utils.metrics import rmsle_metric

from sklearn.pipeline import Pipeline
from sklearn.linear_model import LinearRegression

from xgboost import XGBRegressor
from sklearn.model_selection import ShuffleSplit
from sklearn.metrics import root_mean_squared_log_error
from sklearn.ensemble import StackingRegressor, HistGradientBoostingRegressor
import numpy as np
import xgboost as xgb
from lightgbm import LGBMRegressor
from sklearn.neighbors import KNeighborsRegressor

from catboost import CatBoostRegressor
from utils.categoricals import CategoricalToStringTransformer, CategoricalEncoder, CatogoricalUnknownTransformer
from utils.training import train_with_cross_validation


In [5]:
train_data = pd.read_csv("playground-series-s4e12/train.csv")#.sample(frac=0.1)
test_data = pd.read_csv("playground-series-s4e12/test.csv")

treated_dataset = treat_dataset_pandas_init(train_data, process_as_category=True)
treated_dataset_test = treat_dataset_pandas_init(test_data, process_as_category=True)

non_loged_train_local, non_loged_test_local = joblib.load("cat_non_loged_local.pkl")
non_loged_train, non_loged_test = joblib.load("cat_non_loged.pkl")

treated_dataset['non_log_oof_prediction'] = non_loged_train
treated_dataset['non_log_oof_prediction_local'] = non_loged_train_local
treated_dataset_test['non_log_oof_prediction'] = non_loged_test
treated_dataset_test['non_log_oof_prediction_local'] = non_loged_test_local

X_train = treated_dataset.drop(columns=["Premium Amount"])
y_train = treated_dataset["Premium Amount"]

X_train_non_log = X_train.copy().drop(columns=["non_log_oof_prediction", "non_log_oof_prediction_local"])
X_test_non_log = treated_dataset_test.copy().drop(columns=["non_log_oof_prediction", "non_log_oof_prediction_local"])


In [6]:
#pipeline = build_preprocessing_pipeline(train_data=X_train, imputation_method="simple", binning_method="opti-means", categorical_to_string=True)
#X_train_transformed = pipeline.fit_transform(X_train, y_train)


In [7]:
from sklearn.model_selection import KFold

splitter = KFold(n_splits=5, shuffle=True, random_state=500)


# Lienal 

In [8]:
from sklearn.linear_model import LinearRegression, SGDRegressor


In [9]:
catboost_params = {'learning_rate': 0.30849158244687513, 'iterations': 74, 'depth': 7, 'l2_leaf_reg': 0.9408913103331975, 'min_data_in_leaf': 43, 'subsample': 0.6707321357155331, 'colsample_bylevel': 0.8996427501620363}
categorical_features = [col for col in X_train.columns if X_train[col].dtype == "category"]

In [10]:
catboost_pipeline = Pipeline([
    ("categorical_to_string", CatogoricalUnknownTransformer()),
    ("model",CatBoostRegressor(**catboost_params, cat_features=categorical_features))
])

catboost_results = train_with_cross_validation(X_train, y_train, splitter, catboost_pipeline, preprocess_y=np.log1p, post_process_y=np.expm1, X_test=treated_dataset_test)


Starting fold 1
0:	learn: 1.0670571	total: 330ms	remaining: 24.1s
1:	learn: 1.0524084	total: 537ms	remaining: 19.3s
2:	learn: 1.0443291	total: 688ms	remaining: 16.3s
3:	learn: 1.0400657	total: 879ms	remaining: 15.4s
4:	learn: 1.0379529	total: 1.03s	remaining: 14.2s
5:	learn: 1.0365167	total: 1.18s	remaining: 13.4s
6:	learn: 1.0356575	total: 1.33s	remaining: 12.7s
7:	learn: 1.0351311	total: 1.47s	remaining: 12.2s
8:	learn: 1.0348000	total: 1.62s	remaining: 11.7s
9:	learn: 1.0343624	total: 1.76s	remaining: 11.2s
10:	learn: 1.0339032	total: 1.93s	remaining: 11.1s
11:	learn: 1.0336408	total: 2.08s	remaining: 10.7s
12:	learn: 1.0334812	total: 2.22s	remaining: 10.4s
13:	learn: 1.0332366	total: 2.37s	remaining: 10.1s
14:	learn: 1.0330988	total: 2.52s	remaining: 9.92s
15:	learn: 1.0329658	total: 2.68s	remaining: 9.71s
16:	learn: 1.0328349	total: 2.83s	remaining: 9.51s
17:	learn: 1.0327621	total: 2.98s	remaining: 9.26s
18:	learn: 1.0327019	total: 3.13s	remaining: 9.05s
19:	learn: 1.0325591	tota

In [11]:
catboost_non_log_pipeline = Pipeline([
    ("categorical_to_string", CatogoricalUnknownTransformer()),
    ("model",CatBoostRegressor(**catboost_params, cat_features=categorical_features))
])

catboost_non_log_results = train_with_cross_validation(X_train_non_log, y_train, splitter, catboost_pipeline, X_test=X_test_non_log)


Starting fold 1
0:	learn: 855.8946103	total: 208ms	remaining: 15.2s
1:	learn: 851.1994139	total: 394ms	remaining: 14.2s
2:	learn: 847.6771071	total: 584ms	remaining: 13.8s
3:	learn: 845.3964572	total: 721ms	remaining: 12.6s
4:	learn: 844.0516357	total: 914ms	remaining: 12.6s
5:	learn: 843.1730365	total: 1.04s	remaining: 11.8s
6:	learn: 842.3122279	total: 1.19s	remaining: 11.4s
7:	learn: 841.4815080	total: 1.32s	remaining: 10.9s
8:	learn: 841.0189795	total: 1.47s	remaining: 10.6s
9:	learn: 840.8275530	total: 1.61s	remaining: 10.3s
10:	learn: 840.5059047	total: 1.75s	remaining: 10s
11:	learn: 840.1481398	total: 1.89s	remaining: 9.74s
12:	learn: 839.9948904	total: 2.02s	remaining: 9.49s
13:	learn: 839.9033828	total: 2.16s	remaining: 9.24s
14:	learn: 839.6544089	total: 2.29s	remaining: 9.02s
15:	learn: 839.4616141	total: 2.43s	remaining: 8.82s
16:	learn: 839.3400021	total: 2.59s	remaining: 8.7s
17:	learn: 839.0379479	total: 2.75s	remaining: 8.56s
18:	learn: 838.9282451	total: 2.91s	remaini

In [18]:
lgbm_params = {'boosting_type': 'gbdt', 'objective': 'regression', 'learning_rate': 0.04, 'num_boost_round': 500, 'num_leaves': 169, 'max_depth': 34, 'feature_fraction': 0.5772473473750267, 'bagging_fraction': 0.5660377739630706, 'bagging_freq': 2, 'min_child_samples': 32, 'min_split_gain': 0.04367672923191071, 'min_child_weight': 3.793554610841873, 'lambda_l1': 4.429544342866135, 'lambda_l2': 2.0212763479020808, 'cat_smooth': 26.99333583992606, 'max_bin': 168, 'verbose': -1, 'random_state': 41}

lgbm_pipeline = Pipeline([
    ("model",LGBMRegressor(**lgbm_params))
])

lgbm_results = train_with_cross_validation(X_train, y_train, splitter, lgbm_pipeline, preprocess_y=np.log1p, post_process_y=np.expm1, X_test=treated_dataset_test)
    

Starting fold 1


d:\Development\insurance\venv\lib\site-packages\lightgbm\engine.py:204: UserWarning: Found `num_boost_round` in params. Will use it instead of argument
  _log_warning(f"Found `{alias}` in params. Will use it instead of argument")


Fold 1 - Train: 0.9940004887910749, Validation: 1.0343674011979753
Starting fold 2


d:\Development\insurance\venv\lib\site-packages\lightgbm\engine.py:204: UserWarning: Found `num_boost_round` in params. Will use it instead of argument
  _log_warning(f"Found `{alias}` in params. Will use it instead of argument")


Fold 2 - Train: 0.9947122645758739, Validation: 1.0305670586146372
Starting fold 3


...

d:\Development\insurance\venv\lib\site-packages\lightgbm\engine.py:204: UserWarning: Found `num_boost_round` in params. Will use it instead of argument
  _log_warning(f"Found `{alias}` in params. Will use it instead of argument")


Fold 4 - Train: 0.9935817032566141, Validation: 1.034614999493313
Starting fold 5


d:\Development\insurance\venv\lib\site-packages\lightgbm\engine.py:204: UserWarning: Found `num_boost_round` in params. Will use it instead of argument
  _log_warning(f"Found `{alias}` in params. Will use it instead of argument")


Fold 5 - Train: 0.9954308966534227, Validation: 1.0292341532763623
Cross-validation complete.
Mean validation RMSLE: 1.0330000186777009
Mean train RMSLE: 0.9942709647058237


d:\Development\insurance\venv\lib\site-packages\lightgbm\engine.py:204: UserWarning: Found `num_boost_round` in params. Will use it instead of argument
  _log_warning(f"Found `{alias}` in params. Will use it instead of argument")


In [21]:
lgbm_params = {'boosting_type': 'gbdt', 'objective': 'regression', 'learning_rate': 0.04, 'num_boost_round': 500, 'num_leaves': 169, 'max_depth': 34, 'feature_fraction': 0.5772473473750267, 'bagging_fraction': 0.5660377739630706, 'bagging_freq': 2, 'min_child_samples': 32, 'min_split_gain': 0.04367672923191071, 'min_child_weight': 3.793554610841873, 'lambda_l1': 4.429544342866135, 'lambda_l2': 2.0212763479020808, 'cat_smooth': 26.99333583992606, 'max_bin': 168, 'verbose': -1, 'random_state': 41}

lgbm_pipeline = Pipeline([
    ("model",LGBMRegressor(**lgbm_params))
])

lgbm_results_non_log = train_with_cross_validation(X_train_non_log, y_train, splitter, lgbm_pipeline,  X_test=X_test_non_log)
    

Starting fold 1


d:\Development\insurance\venv\lib\site-packages\lightgbm\engine.py:204: UserWarning: Found `num_boost_round` in params. Will use it instead of argument
  _log_warning(f"Found `{alias}` in params. Will use it instead of argument")


Fold 1 - Train: 1.11387080166294, Validation: 1.1332342476402129
Starting fold 2


d:\Development\insurance\venv\lib\site-packages\lightgbm\engine.py:204: UserWarning: Found `num_boost_round` in params. Will use it instead of argument
  _log_warning(f"Found `{alias}` in params. Will use it instead of argument")


Fold 2 - Train: 1.1146778493149874, Validation: 1.1294018920447808
Starting fold 3


...

d:\Development\insurance\venv\lib\site-packages\lightgbm\engine.py:204: UserWarning: Found `num_boost_round` in params. Will use it instead of argument
  _log_warning(f"Found `{alias}` in params. Will use it instead of argument")


Fold 4 - Train: 1.1135932864306495, Validation: 1.133941817862566
Starting fold 5


d:\Development\insurance\venv\lib\site-packages\lightgbm\engine.py:204: UserWarning: Found `num_boost_round` in params. Will use it instead of argument
  _log_warning(f"Found `{alias}` in params. Will use it instead of argument")


Fold 5 - Train: 1.1150491846586148, Validation: 1.1272374187655532
Cross-validation complete.
Mean validation RMSLE: 1.132118497501275
Mean train RMSLE: 1.1140792448451868


d:\Development\insurance\venv\lib\site-packages\lightgbm\engine.py:204: UserWarning: Found `num_boost_round` in params. Will use it instead of argument
  _log_warning(f"Found `{alias}` in params. Will use it instead of argument")


In [22]:
xgboost_params = {'learning_rate': 0.04656560304624032, 'max_depth': 10, 'min_child_weight': 50, 'subsample': 0.9730484625285342, 'colsample_bytree': 0.5743561244230219, 'gamma': 7.9578377008338235, 'lambda': 9.268384283962487, 'alpha': 9.9663808717552, 'n_estimators': 754}

xgboost_pipeline = Pipeline([
    #("categorical_encoder", CategoricalEncoder()),
    ("model",xgb.XGBRegressor(enable_categorical=True, **xgboost_params, device="cuda", tree_method="hist"))
])

xgboost_results = train_with_cross_validation(X_train, y_train, splitter, xgboost_pipeline, preprocess_y=np.log1p, post_process_y=np.expm1, X_test=treated_dataset_test)

Starting fold 1
Fold 1 - Train: 1.0236045947632795, Validation: 1.0323939928240387
Starting fold 2
Fold 2 - Train: 1.024410623014631, Validation: 1.0290582343357149
Starting fold 3
Fold 3 - Train: 1.022992146444883, Validation: 1.0346463697514314
Starting fold 4
Fold 4 - Train: 1.0232958456399106, Validation: 1.0329454703059728
Starting fold 5
Fold 5 - Train: 1.0247934589506915, Validation: 1.0273757790878009
Cross-validation complete.
Mean validation RMSLE: 1.0312839692609916
Mean train RMSLE: 1.0238193337626793


In [27]:
xgboost_low_complexity_params = {'learning_rate': 0.05, 'max_depth': 4, 'min_child_weight': 50, 'subsample': 0.8, 'colsample_bytree': 0.7,  'n_estimators': 100}

xgboost_low_complexity_pipeline = Pipeline([
    #("categorical_encoder", CategoricalEncoder()),
    ("model",xgb.XGBRegressor(enable_categorical=True, **xgboost_params, device="cuda", tree_method="hist"))
])

xgboost_low_complexity_results = train_with_cross_validation(X_train, y_train, splitter, xgboost_low_complexity_pipeline, preprocess_y=np.log1p, post_process_y=np.expm1, X_test=treated_dataset_test)

Starting fold 1
Fold 1 - Train: 1.0236045947632795, Validation: 1.0323939928240387
Starting fold 2
Fold 2 - Train: 1.024410623014631, Validation: 1.0290582343357149
Starting fold 3
Fold 3 - Train: 1.022992146444883, Validation: 1.0346463697514314
Starting fold 4
Fold 4 - Train: 1.0232958456399106, Validation: 1.0329454703059728
Starting fold 5
Fold 5 - Train: 1.0247934589506915, Validation: 1.0273757790878009
Cross-validation complete.
Mean validation RMSLE: 1.0312839692609916
Mean train RMSLE: 1.0238193337626793


In [26]:
xgboost_params = {'learning_rate': 0.04656560304624032, 'max_depth': 10, 'min_child_weight': 50, 'subsample': 0.9730484625285342, 'colsample_bytree': 0.5743561244230219, 'gamma': 7.9578377008338235, 'lambda': 9.268384283962487, 'alpha': 9.9663808717552, 'n_estimators': 754}

xgboost_non_log_pipeline = Pipeline([
    #("categorical_encoder", CategoricalEncoder()),
    ("model",xgb.XGBRegressor(enable_categorical=True, **xgboost_params, device="cuda", tree_method="hist"))
])

xgboost_non_log_results = train_with_cross_validation(X_train_non_log, y_train, splitter, xgboost_non_log_pipeline, preprocess_y=np.log1p, post_process_y=np.expm1, X_test=X_test_non_log)

Starting fold 1
Fold 1 - Train: 1.0387890654783232, Validation: 1.0463906057796561
Starting fold 2
Fold 2 - Train: 1.0397416807812168, Validation: 1.0428227899083957
Starting fold 3
Fold 3 - Train: 1.038083145581992, Validation: 1.0491916860512465
Starting fold 4
Fold 4 - Train: 1.0386444837291342, Validation: 1.0469797152387068
Starting fold 5
Fold 5 - Train: 1.0400446392732245, Validation: 1.0423748114069145
Cross-validation complete.
Mean validation RMSLE: 1.045551921676984
Mean train RMSLE: 1.0390606029687781


# Blending

In [29]:
lineal_results = joblib.load("lineal_results.pkl")


In [82]:
final_results = pd.DataFrame({
    #"non_log_oof_prediction": treated_dataset['non_log_oof_prediction'],
    #"non_log_oof_prediction_local": treated_dataset['non_log_oof_prediction_local'],
    "catboost_oof_prediction": catboost_results['oof_predictions'],
    "catboost_non_log_oof_prediction": catboost_non_log_results['oof_predictions'],
    "xgboost_oof_prediction": xgboost_results['oof_predictions'],
    "xgboost_low_complexity_oof_prediction": xgboost_low_complexity_results['oof_predictions'],
    "xgboost_non_log_oof_prediction": xgboost_non_log_results['oof_predictions'],
    "lgbm_oof_prediction": lgbm_results['oof_predictions'],
    "lgbm_non_log_oof_prediction": lgbm_results_non_log['oof_predictions'],
    "lineal_oof_non_log_prediction": lineal_results["non_log"]["oof_predictions"],
    "lineal_oof_log_prediction": lineal_results["log"]["oof_predictions"],
})

final_test_results = pd.DataFrame({
    #"non_log_oof_prediction": treated_dataset['non_log_oof_prediction'],
    #"non_log_oof_prediction_local": treated_dataset['non_log_oof_prediction_local'],
    "catboost_oof_prediction": catboost_results['fold_test_predictions'].mean(axis=1),
    "catboost_non_log_oof_prediction": catboost_non_log_results['fold_test_predictions'].mean(axis=1),
    "xgboost_oof_prediction": xgboost_results['fold_test_predictions'].mean(axis=1),
    "xgboost_low_complexity_oof_prediction": xgboost_low_complexity_results['fold_test_predictions'].mean(axis=1),
    "xgboost_non_log_oof_prediction": xgboost_non_log_results['fold_test_predictions'].mean(axis=1),
    "lgbm_oof_prediction": lgbm_results['fold_test_predictions'].mean(axis=1),
    "lgbm_non_log_oof_prediction": lgbm_results_non_log['fold_test_predictions'].mean(axis=1),
    "lineal_oof_non_log_prediction": lineal_results["non_log"]["fold_test_predictions"].mean(axis=1),
    "lineal_oof_log_prediction": lineal_results["log"]["fold_test_predictions"].mean(axis=1)
})

In [91]:
#for col in final_test_results.columns:
#    final_results[f"{col}_log_trfm"] = np.log1p(final_results[col])
    final_test_results[f"{col}_log_trfm"] = np.log1p(final_test_results[col])


IndentationError: unexpected indent (2646264478.py, line 3)

In [92]:
X_train_blending = final_results.merge(X_train[["Annual Income",'Credit Score','Policy Start Year','Week','Group']], left_index=True, right_index=True).copy()
X_test_blending = final_test_results.merge(treated_dataset_test[["Annual Income",'Credit Score','Policy Start Year','Week','Group']], left_index=True, right_index=True).copy()


In [93]:
from sklearn.linear_model import BayesianRidge


In [94]:
catboost_params = {'learning_rate': 0.1, 'iterations': 1000, 'depth': 12, 'l2_leaf_reg': 15, 'min_data_in_leaf': 15, 'subsample': 0.9, 'colsample_bylevel': 0.9}

final_models = [] 
for train_idx, val_idx in splitter.split(X_train_blending):
    print("Starting")
    # Prepare data for LightGBM
    X_train_fold, X_val_fold = X_train_blending.iloc[train_idx], X_train_blending.iloc[val_idx]
    y_train_fold, y_val_fold = y_train.iloc[train_idx], y_train.iloc[val_idx]
    
    y_train_fold = np.log1p(y_train_fold)
    y_val_fold = np.log1p(y_val_fold)
    
    #full_pipeline = Pipeline([
        #("categorical_encoder", CategoricalEncoder()),
       # ("model",xgb.XGBRegressor(enable_categorical=True, **xgboost_final_params))
    #])
    # full_pipeline.fit(X_train_fold, y_train_fold)
    categorical_features = [col for col in X_train_fold.columns if X_train_fold[col].dtype == "category"]
    full_pipeline = Pipeline([
        ("categorical_to_string", CatogoricalUnknownTransformer()),
        ("model",CatBoostRegressor(**catboost_params, cat_features=categorical_features))
    ])  
    
    full_pipeline.fit(X_train_fold, y_train_fold, model__eval_set=(X_val_fold, y_val_fold), model__verbose=5, model__early_stopping_rounds=5)
    final_models.append(full_pipeline)

    train_score = root_mean_squared_log_error(np.expm1(full_pipeline.predict(X_train_fold)), np.expm1(y_train_fold))
    val_score = root_mean_squared_log_error(np.expm1(full_pipeline.predict(X_val_fold)), np.expm1(y_val_fold))
    
    #train_score = root_mean_squared_log_error(full_pipeline.predict(X_train_fold), y_train_fold)
    #val_score = root_mean_squared_log_error(full_pipeline.predict(X_val_fold),y_val_fold)

    print(f"Train: {train_score}, Test: {val_score}")

Starting
0:	learn: 1.0839350	test: 1.0842967	best: 1.0842967 (0)	total: 652ms	remaining: 10m 51s
5:	learn: 1.0500659	test: 1.0511215	best: 1.0511215 (5)	total: 4.17s	remaining: 11m 30s
10:	learn: 1.0375361	test: 1.0390153	best: 1.0390153 (10)	total: 7.68s	remaining: 11m 30s
15:	learn: 1.0328692	test: 1.0347128	best: 1.0347128 (15)	total: 11.2s	remaining: 11m 26s
20:	learn: 1.0310482	test: 1.0331797	best: 1.0331797 (20)	total: 14.6s	remaining: 11m 19s
25:	learn: 1.0302565	test: 1.0326286	best: 1.0326286 (25)	total: 17.2s	remaining: 10m 46s
30:	learn: 1.0297995	test: 1.0324203	best: 1.0324203 (30)	total: 20.7s	remaining: 10m 45s
35:	learn: 1.0292623	test: 1.0323283	best: 1.0323259 (34)	total: 24.2s	remaining: 10m 48s
40:	learn: 1.0289288	test: 1.0322849	best: 1.0322849 (40)	total: 27.6s	remaining: 10m 45s
45:	learn: 1.0286203	test: 1.0322548	best: 1.0322548 (45)	total: 31.1s	remaining: 10m 43s
50:	learn: 1.0283119	test: 1.0322579	best: 1.0322548 (45)	total: 34s	remaining: 10m 32s
Stopped

In [95]:
catboost_params = {'learning_rate': 0.1, 'iterations': 50, 'depth': 12, 'l2_leaf_reg': 15, 'min_data_in_leaf': 15, 'subsample': 0.9, 'colsample_bylevel': 0.9}
full_pipeline = Pipeline([
    ("categorical_to_string", CatogoricalUnknownTransformer()),
    ("model",CatBoostRegressor(**catboost_params, cat_features=categorical_features))
])  
    
full_pipeline = full_pipeline.fit(X_train_blending, np.log1p(y_train))

0:	learn: 1.0840367	total: 615ms	remaining: 30.1s
1:	learn: 1.0743764	total: 1.24s	remaining: 29.8s
2:	learn: 1.0664155	total: 1.82s	remaining: 28.5s
3:	learn: 1.0598967	total: 2.42s	remaining: 27.9s
4:	learn: 1.0546136	total: 3.03s	remaining: 27.3s
5:	learn: 1.0502523	total: 3.64s	remaining: 26.7s
6:	learn: 1.0466890	total: 4.24s	remaining: 26.1s
7:	learn: 1.0437868	total: 4.78s	remaining: 25.1s
8:	learn: 1.0414299	total: 5.37s	remaining: 24.5s
9:	learn: 1.0394962	total: 5.96s	remaining: 23.8s
10:	learn: 1.0378966	total: 6.53s	remaining: 23.2s
11:	learn: 1.0365945	total: 7.08s	remaining: 22.4s
12:	learn: 1.0355149	total: 7.7s	remaining: 21.9s
13:	learn: 1.0346655	total: 8.27s	remaining: 21.3s
14:	learn: 1.0339399	total: 8.83s	remaining: 20.6s
15:	learn: 1.0333658	total: 9.42s	remaining: 20s
16:	learn: 1.0328795	total: 9.97s	remaining: 19.4s
17:	learn: 1.0324433	total: 10.6s	remaining: 18.8s
18:	learn: 1.0320696	total: 11.2s	remaining: 18.3s
19:	learn: 1.0317822	total: 11.8s	remaining:

In [96]:
test_predictions = np.expm1(full_pipeline.predict(X_test_blending))
test_predictions = np.clip(test_predictions, 20, 4000)


In [97]:
submission = pd.DataFrame({"id": test_data["id"], "Premium Amount": test_predictions})
submission.to_csv("submission_tree_blending_catboost_final.csv", index=False)


In [98]:
submission

,id,Premium Amount
0,1200000,846.618705
1,1200001,907.221041
2,1200002,823.056176
3,1200003,813.375268
4,1200004,807.112691
...,...,...
799995,1999995,976.413858
799996,1999996,725.420027
799997,1999997,850.130167
799998,1999998,897.024159
